In [31]:
import pandas as pd
import os
import ast
import re
 
# List of file names
bath = "Dataset\\"
file_names = ['bangalore_cars.xlsx', 'chennai_cars.xlsx', 'delhi_cars.xlsx',
              'hyderabad_cars.xlsx', 'jaipur_cars.xlsx', 'kolkata_cars.xlsx']
dfs = []
 
for file in file_names:
    df = pd.read_excel(f"{bath}{file}")
    state = file.split('_')[0]
    df['state'] = state
   
    dfs.append(df)
 
merged_df = pd.concat(dfs, ignore_index=True)
 
# merged_df.to_excel('merged_cars_data.xlsx', index=False)
 
print("Files merged successfully. Output saved as 'merged_cars_data.xlsx'")

Files merged successfully. Output saved as 'merged_cars_data.xlsx'


In [29]:
merged_df

,new_car_detail,new_car_overview,new_car_feature,new_car_specs,car_links,state
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,bangalore
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...,bangalore
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,bangalore
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...,bangalore
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,bangalore
...,...,...,...,...,...,...
8364,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,kolkata
8365,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,kolkata
8366,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,kolkata
8367,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,kolkata


In [32]:
def convert_to_dict(row):
    try:
        a = ast.literal_eval(row)
        return a
    except ValueError:
        return None
    

def first(car_data):
    df = {}
    for row in car_data:
        for key in row:
            if key not in df:
                df[key] = []
            df[key].append(row[key])
    df = pd.DataFrame(df)
    return df

def top(car_data):
        def extract_key_value_pairs(d):
            flat_dict = {}
            try:
                if 'top' in d:
                    for item in d['top']:
                        key = item.get('key')  # Extracting the key
                        value = item.get('value') # Extracting the value
                        if key:
                            flat_dict[key] = value  # Use key as column name, value as entry
            except:
                pass
            return flat_dict
        flattened_data = [extract_key_value_pairs(d) for d in car_data if d is not None]
        df = pd.DataFrame(flattened_data)
        return df


def data(car_data):
    def extract_key_value_pairs(d):
        flat_dict = {}
        try:
            # Proceed only if 'data' exists in the dictionary
            if d is not None and 'data' in d:
                for section in d['data']:
                    if 'list' in section:
                        for item in section['list']:
                            key = item.get('key')  # Using .get to avoid KeyError
                            value = item.get('value')
                            if key and value:  # Ensure both key and value exist
                                flat_dict[key] = value
        except Exception as e:
            # Log or handle exception if needed
            print(f"Error processing row: {e}")
        return flat_dict

    # Filter out None entries
    flattened_data = [extract_key_value_pairs(d) for d in car_data if d is not None]
    df = pd.DataFrame(flattened_data)
    return df



In [33]:
car_columns = ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']

for columns in car_columns:
    merged_df[columns] = merged_df[columns].apply(convert_to_dict)


final_df = pd.DataFrame()

q = pd.DataFrame()
for i in car_columns:
    w = first(merged_df[i])

    if w is None:
        continue
    columns = w.columns

    if "top" in columns:
        w = top(merged_df[i])

    if "data" in columns:
        q = data(merged_df[i])

    final_df = pd.concat([final_df, w], axis=1, ignore_index=False)
    if not q.empty:
        final_df = pd.concat([final_df, q], axis=1, ignore_index=False)
final_df['city'] = merged_df["state"]
final_df['car_links'] = merged_df['car_links']

In [17]:
pd.set_option('display.max_columns', None)
final_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture,Mileage,Engine,Max Power,Torque,Seats,Wheel Size,Color,Engine Type,Displacement,Max Power,Max Torque,No of Cylinder,Values per Cylinder,Value Configuration,Fuel Suppy System,BoreX Stroke,Compression Ratio,Turbo Charger,Super Charger,Length,Width,Height,Wheel Base,Front Tread,Rear Tread,Kerb Weight,Gross Weight,Gear Box,Drive Type,Seating Capacity,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Alloy Wheel Size,Ground Clearance Unladen,city
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,3979,VXI,₹ 4 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,2015,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",KA51,Third Owner,998 cc,Manual,2015.0,23.1 kmpl,998 CC,67.04bhp,90Nm,5,NaN,White,K10B Engine,998,67.04bhp@6000rpm,90Nm@3500rpm,3.0,4.0,DOHC,MPFi,73 X 82 mm,11.0:1,No,No,3715mm,1635mm,1565mm,2425mm,1420mm,1410mm,835kg,1250kg,5 Speed,FWD,5,Power,4.7 metres,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,NaN,bangalore
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,6087,1.5 Petrol Titanium BSIV,₹ 8.11 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,Feb 2018,Comprehensive,Petrol,5 Seats,"32,706 Kms",KA05,Second Owner,1497 cc,Manual,2018.0,17 kmpl,1497 CC,121.31bhp,150Nm,5,16,White,Ti-VCT Petrol Engine,1497,121.31bhp@6500rpm,150Nm@4500rpm,3.0,4.0,DOHC,Direct Injection,79 X 76.5 mm,11.0:1,No,No,3998mm,1765mm,1647mm,2519mm,NaN,NaN,1242Kg,1660Kg,5 Speed,FWD,5,Power,5.3 metres,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16,NaN,bangalore
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,2983,1.2 Revotron XZ,₹ 5.85 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,Sept 2018,Comprehensive,Petrol,5 Seats,"11,949 Kms",KA03,First Owner,1199 cc,Manual,2018.0,23.84 kmpl,1199 CC,84bhp,114Nm,5,14,Red,Revotron Engine,1199,84bhp@6000rpm,114Nm@3500rpm,3.0,4.0,DOHC,MPFi,77 X 85.8 mm,10.8:1,No,No,3746mm,1647mm,1535mm,2400mm,1400mm,1420mm,1012kg,NaN,5 Speed,FWD,5,Power,4.9 meters,Disc,Drum,150 kmph,14.3 Seconds,Tubeless,5,242-litres,14,NaN,bangalore
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,1867,1.2 Kappa S Option,₹ 4.62 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,Dec 2014,Comprehensive,Petrol,5 Seats,"17,794 Kms",KA53,First Owner,1197 cc,Manual,2014.0,19.1 kmpl,1197 CC,81.86bhp,113.75Nm,5,14,Others,Kappa VTVT Petrol Engine,1197,81.86bhp@6000rpm,113.75Nm@4000rpm,4.0,4.0,DOHC,Direct Injection,NaN,NaN,No,No,3995mm,1660mm,1520mm,2425mm,1479mm,1493mm,1180,NaN,5 Speed,FWD,5,Power,4.7 metres,Disc,Drum,172km/hr,14.2 Seconds,"Tubeless,Radial",4,407-litres,14,NaN,bangalore
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,4277,DDiS 200 Zeta,₹ 7.90 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,2015,Third Party insurance,Diesel,5 Seats,"60,000 Kms",KA04,First Owner,1248 cc,Manual,2015.0,23.65 kmpl,1248 CC,88.5bhp,200Nm,5,16,Gray,DDiS 200 Diesel Engine,1248,88.5bhp@4000rpm,200Nm@1750rpm,4.0,4.0,DOHC,NaN,69.6 x 82 mm,NaN,Yes,No,4300mm,1785mm,1595mm,2600mm,NaN,NaN,1230Kg,1670kg,5 Speed,FWD,5,Power,5.2 meters,Ventilated Disc,Solid Disc,190 Kmph,12 Seconds,"Tubeless,Radial",5,353-litres,16,NaN,bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,8682,ZXI BSVI,₹ 5.10 Lakh,

In [34]:
df = final_df.copy()
def clean_km(km):
    km = int(km.replace(',','').strip())
    return km
df['km']=df['km'].apply(clean_km)


df['ownerNo']=df['ownerNo'].astype(int)
df['centralVariantId']=df['centralVariantId'].astype(int)
df['modelYear']=df['modelYear'].astype(int)


def clean_price(price):
    price = price.replace(',','').replace('₹','').strip()
    try:
        if 'Lakh' in price:
            price = float(price.replace('Lakh','').strip()) * 100000
        elif 'Crore' in price:
            price = float(price.replace('Crore','').strip()) * 10000000
        else:
            price = float(price)
    except ValueError:
        return None
    
    return price

df['price']=df['price'].apply(clean_price)

df['price']=df['price'].astype(int)

def clean_ed(eng):
    if isinstance(eng, str):  # Check if eng is a string
        eng = int(eng.replace('CC', '').strip())  # Clean the string
        return eng
    return eng 

df['Engine']=df['Engine'].apply(clean_ed)

df = df.dropna(subset=['Engine'])

df['Engine']=df['Engine'].astype(int)


def clean_ml(Mileage):

    if isinstance(Mileage, str):  # Check if Mileage is a string
        Mileage = Mileage.replace('kmpl', '').replace('km/kg','').strip()
        Mileage = float(Mileage)
    return Mileage
df['Mileage']=df['Mileage'].apply(clean_ml)
# df = df.dropna(subset='Mileage')
df["Mileage"].fillna(df["Mileage"].mean(), inplace=True)
df["Mileage"].replace(140.,df["Mileage"].mean(), inplace=True)
 



df = df.dropna(subset=['Seating Capacity'])
df = df.drop(columns=["Seats"])



def clean_ins(a):
    if a == "Third Party insurance":
        return "Third Party"
    elif a == "2":
        return None
    elif a == "1":
        return None
    else:
        return a
df['Insurance Validity'] = df['Insurance Validity'].apply(clean_ins)
df = df.dropna(subset = ['Insurance Validity'])



def st(row):
    row = str(row)
    row = row.replace(" ", "")
    return row
df['Gear Box'] = df['Gear Box'].apply(st)
def clean_gearbox(data):
    cleaned_data = []
    for entry in data:
        # Normalize to lowercase
        entry = entry.lower()

        # Replace various "speed" patterns with a standard format
        entry = re.sub(r'(\d+)[- ]?speed', r'\1-speed', entry)
        
        # Replace specific known gearbox types
        entry = re.sub(r'cvt|ecvt|ivt', 'cvt', entry)
        entry = re.sub(r'(automatic|auto)', 'automatic', entry)
        entry = re.sub(r'(manual)', 'manual', entry)
        
        # Remove redundant terms like "shift", "transmission", "sport"
        entry = re.sub(r'(sport|shift|transmission|gearbox|paddleshifters)', '', entry)
        
        # Clean any leftover non-standard characters or spaces
        entry = re.sub(r'[^\w-]', '', entry)
        entry = re.sub(r'-+', '-', entry)  # Replace multiple hyphens with a single one
        entry = re.sub(r'\s+', ' ', entry).strip()  # Remove extra spaces

        # Add cleaned entry to list
        cleaned_data.append(entry)
    
    return cleaned_data

# Clean the data
df['Gear Box_new'] = clean_gearbox(df['Gear Box'])



df_1 = pd.DataFrame(df['Max Power'])
df_1.columns = ['Max Power (bhp)', 'Max Power (bhp@rpm)']
df = df.drop(columns=["Max Power"])
df['Max Power (bhp)'] = df_1['Max Power (bhp)']
df['Max Power (bhp@rpm)'] = df_1['Max Power (bhp@rpm)']



def extract_integer(value):
    # Use regular expression to find all numeric parts of the string
    result = re.findall(r'\d+', str(value))
    if result:
        return int(result[0])  # Convert the first numeric part to integer
    return None  # Return None if no numeric part is found

# Apply the function to the column
df['Max Power (int)'] = df['Max Power (bhp)'].apply(extract_integer)


df['Torque_int'] = df['Torque'].apply(extract_integer)

df['Length_new'] = df['Length'].apply(extract_integer)

df['Width_new'] = df['Width'].apply(extract_integer)

df['Wheel Base_new'] = df['Wheel Base'].apply(extract_integer)

df['Height_new'] = df['Height'].apply(extract_integer)


C:\Users\infog\AppData\Local\Temp\ipykernel_1128\2715879014.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Mileage"].fillna(df["Mileage"].mean(), inplace=True)
C:\Users\infog\AppData\Local\Temp\ipykernel_1128\2715879014.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
 
def co(column_data, price,column_):
    
    data = pd.DataFrame()
    data[column_] = column_data
    data['price'] = price
    f = data[column_].isnull().sum()
    data = data.dropna()
    le = LabelEncoder()
    # Check if the column is non-numerical
    if data[column_].dtype == 'object':
        data[column_] = le.fit_transform(data[column_].astype(str))
    
    correlation = data[i].corr(data['price'])
    return correlation, f
 
column_s = df.columns
# df_cleaned = df.drop(columns=["Seats"])

for i in column_s:
    t = co(df[i],df["price"],i)
    print(i,t)

c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_bas

it (nan, 0)
ft (-0.18122742477525258, 0)
bt (0.26555472599220914, 0)
km (-0.07728285160989969, 0)
transmission (-0.4228634067334418, 0)
ownerNo (-0.12020438136371243, 0)
owner (-0.12020438136371243, 0)
oem (-0.03457577678051242, 0)
model (-0.008156998798375375, 0)
modelYear (0.29989508731788245, 0)
centralVariantId (0.18331949245727008, 0)
variantName (-0.037503194942651606, 0)
price (1.0, 0)
priceActual (-0.11031266881046592, 0)
priceSaving (nan, 0)
priceFixedText (nan, 8351)
trendingText (nan, 0)
Registration Year (0.07092614822770349, 50)
Insurance Validity (-0.14890406194715847, 0)
Fuel Type (-0.18122742477525258, 0)
Kms Driven (-0.0691382460443992, 2)
RTO (-0.05793759582054702, 885)
Ownership (-0.10916921271617927, 32)
Engine Displacement (0.14688848559886258, 0)
Transmission (-0.4228634067334418, 0)
Year of Manufacture (0.3023710297770043, 18)
Mileage (-0.2854776945507024, 0)
Engine (0.5366023283427888, 0)
Torque (0.22297094763226688, 53)
Wheel Size (0.4103788837364381, 2973)
Col

In [36]:
data = pd.DataFrame()
data['ft'] = df['ft'] # -0.18134117171709702
data['bt'] = df['bt'] # 0.26557820440500396
data['km'] = df['km'] # -0.07731156737902267
data['centralVariantId'] = df['centralVariantId']
data['transmission'] = df['transmission'] # -0.4225535727873822
data['ownerNo'] = df['ownerNo'] # -0.120160374550546
data['oem'] = df['oem'] # -0.034502979318195424
data['modelYear'] = df['modelYear'] # 0.29985527995117445
data['price'] = df['price'] # 
data['Year of Manufacture'] = df['Year of Manufacture']
data['Insurance Validity'] = df['Insurance Validity'] # -0.14890406194715847
data['Mileage'] = df['Mileage'] # -0.33459405482007415
data['Engine'] = df['Engine'] # 0.5366301966387229
data['city'] = df['city'] # -0.07576194593563172
data['Seating Capacity'] = df['Seating Capacity'] # 0.09265830372537098
data['Gear Box'] = df['Gear Box_new'] # 0.5384508750941248
data['No of Cylinder'] = df['No of Cylinder'] # 0.3848274073406987
data['Max Power'] = df['Max Power (int)'] # -0.20280693566606953
data['Torque'] = df['Torque_int'] # 0.22297094763226688
data["Length"] = df['Length_new'] # 0.5875880269899488
data['Width'] = df['Width_new'] # 0.6401480537843525
data['Height'] = df['Height_new']
data['car_links']= df['car_links']


data.isna().sum()
data["Width"].fillna(data["Width"].mean(), inplace=True)
data["Length"].fillna(data["Length"].mean(), inplace=True)
data["Torque"].fillna(data["Torque"].mean(), inplace=True)
data["Max Power"].fillna(data["Max Power"].mean(), inplace=True)
data["No of Cylinder"].fillna(data["No of Cylinder"].mean(), inplace=True)
data["Year of Manufacture"].fillna(data["Year of Manufacture"].mean(), inplace=True)
data["Height"].fillna(data["Height"].mean(), inplace=True)
data = data.dropna(subset='Gear Box')
data = data.dropna(subset='bt')

# ft, km, transmission, owner no, Oem, ModelYear, price, insurance Validity, Mileage, engine, city, Seating Capacity

C:\Users\infog\AppData\Local\Temp\ipykernel_1128\4264805021.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Width"].fillna(data["Width"].mean(), inplace=True)
C:\Users\infog\AppData\Local\Temp\ipykernel_1128\4264805021.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [37]:
data.to_csv("Dataset\\data.csv",index=False)

In [39]:
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Read the CSV file into a DataFrame
data = pd.read_csv("Dataset/data.csv")

# Step 2: Create a connection to MySQL using SQLAlchemy
engine = create_engine('mysql+mysqlconnector://root:@localhost/car_price')

# Step 3: Upload DataFrame to MySQL in chunks to avoid connection issues
data.to_sql('car_details', con=engine, if_exists='replace', index=False, chunksize=1000)

print("Data uploaded successfully!")


Data uploaded successfully!
